In [110]:
# importação das bibliotecas utilizadas
import numpy as np
import os
import pandas as pd

In [111]:
# carregando os filmes indicados ou vencedores do Oscar em um dataframe
oscar_movies = pd.read_csv('/kaggle/input/the-oscar-award/the_oscar_award.csv')

# examinando as características do dataframe
oscar_movies.info()
oscar_movies.describe()

In [112]:
# A única coluna com itens faltantes é a coluna film.
# Vamos explorar as linhas em que faltam esses dados para entender o porquê,
# e em quais categorias isso está ocorrendo

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()].head()

In [113]:
# Alguns desses casos se tratam de prêmios honorários, humanitários e memoriais - 
# ou seja, não relacionados a filmes específicos. Podemos desconsiderar essas linhas

rows_to_drop = oscar_movies[
    (oscar_movies['category'] == 'HONORARY AWARD') | 
    (oscar_movies['category'] == 'SPECIAL AWARD') | 
    (oscar_movies['category'] == 'IRVING G. THALBERG MEMORIAL AWARD') | 
    (oscar_movies['category'] == 'JEAN HERSHOLT HUMANITARIAN AWARD') | 
    (oscar_movies['category'] == 'SPECIAL ACHIEVEMENT AWARD')
].index

oscar_movies = oscar_movies.drop(rows_to_drop)

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()].head()

In [114]:
# As linhas restantes com dados faltantes na coluna filme são de categorias 
# que precisam necessariamente estar ligadas a um filme.

# Para duas dessas categorias, ligadas a filmes estrangeiros, o nome do filme parece constar
# na coluna 'name'

for row in oscar_movies[
    (oscar_movies['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') | 
    (oscar_movies['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')
].iterrows():
    print(row[1][4])

# em todos esses casos, com exceção do último, o nome do filme consta na coluna 'name',
# seguido de hífen. Podemos aplicar a mesma regra para todos os casos, e transpor o nome do 
# filme para a coluna film
oscar_movies['film'] = oscar_movies.apply(
    lambda row: row['name'].split('-')[0] if row['category'] in ('SPECIAL FOREIGN LANGUAGE FILM AWARD', 'HONORARY FOREIGN LANGUAGE FILM AWARD') 
    else row['film'],
    axis=1
)

# validando que as alterações foram feitas corretamente
oscar_movies[(oscar_movies['category'] == 'SPECIAL FOREIGN LANGUAGE FILM AWARD') | (oscar_movies['category'] == 'HONORARY FOREIGN LANGUAGE FILM AWARD')]

In [115]:
# Com os dados dessas categorias corrigidos,
# é hora de checar os dados faltantes que ainda restam

print(oscar_movies[oscar_movies['film'].isnull()]['category'].unique())
oscar_movies[oscar_movies['film'].isnull()].head()

In [116]:
# Outras linhas com dados faltantes na coluna 'film' são de obtenção difícil ou imprecisa: 
# um assistente de direção, por exemplo, pode ter tido mais de um filme lançado num mesmo 
# ano, o que faria a checagem dos dados restantes muito trabalhosa.
# Essas colunas serão ignoradas.

rows_to_drop = oscar_movies[
    (
        (oscar_movies['category'] == 'ENGINEERING EFFECTS') | 
        (oscar_movies['category'] == 'WRITING (Title Writing)') | 
        (oscar_movies['category'] == 'SOUND RECORDING') | 
        (oscar_movies['category'] == 'ASSISTANT DIRECTOR')
    ) & (oscar_movies['film'].isnull())
].index

oscar_movies = oscar_movies.drop(rows_to_drop)

oscar_movies.info()

In [117]:
# Um fato que pode ser observado analisando esses dados restantes é que algumas das 
# categorias tiveram mudanças de nomes - como foi o caso na premiação de filme de língua 
# estrangeira. Já outras categorias desapareceram da premiação ("melhor filme em preto e 
# branco") ou foram incluídas.

# A categoria mais recente incluída na premiação é a de melhor filme de animação, criada em 
# 2002. Por isso, iremos desconsiderar filmes anteriores a essa data.

rows_to_drop = oscar_movies[(oscar_movies['year_ceremony'] < 2002)].index
oscar_movies = oscar_movies.drop(rows_to_drop)

# Iremos desconsiderar ainda a premiação de edição de som (sound editing), 
# que foi descontinuada em 2019, a única removida após o ano de 2002.
rows_to_drop = oscar_movies[(oscar_movies['category'] == 'SOUND EDITING')].index
oscar_movies = oscar_movies.drop(rows_to_drop)

list(oscar_movies['category'].unique())

In [118]:
# Algumas dessas categorias também precisaram de consolidação, já que seus nomes 
# aparecem de formas diferentes. Foram utilizados os nomes mais recentes.

category_renaming = {
    'FOREIGN LANGUAGE FILM': 'INTERNATIONAL FEATURE FILM',
    'ART DIRECTION': 'PRODUCTION DESIGN',
    'WRITING (Screenplay Based on Material Previously Produced or Published)': 'WRITING (Adapted Screenplay)',
    'WRITING (Screenplay Written Directly for the Screen)': 'WRITING (Original Screenplay)',
    'SOUND MIXING': 'SOUND',
    'MAKEUP': 'MAKEUP AND HAIRSTYLING',
}

oscar_movies['category'] = oscar_movies.apply(
    lambda row: category_renaming[row['category']]
    if row['category'] in category_renaming 
    else row['category'],
    axis=1
)

# Checando que agora temos as 23 categorias atuais do Oscar
list(oscar_movies['category'].unique())

In [119]:
# Agora precisamos unir os dados do dataset do Oscar ao dataset de metadados de filmes.

# carregando os metadados de todos os filmes
movies_metadata = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')

# examinando as características do dataframe
movies_metadata.info()
movies_metadata.describe()